In [1]:
import pandas as pd
import os
import textblob  
from textblob import TextBlob

In [2]:
file_DML="reviews_DML_US_GP.csv"
file_DC= "reviews_DC_US_GP.csv"
file_DV="reviews_DV_US_GP.csv"

In [3]:
DML=pd.read_csv(f"data/{file_DML}",sep="\t")
DC=pd.read_csv(f"data/{file_DC}",sep="\t")
DV=pd.read_csv(f"data/{file_DV}",sep="\t")

In [4]:
print(DML.head)
print(DC.head)
print(DV.head)


<bound method NDFrame.head of       Country  Rating        Date Version          Username Title  \
0          US       5  2023-02-28  7.2.0e       Ayman Ahmed   NaN   
1          US       1  2023-02-27  7.2.0e      anh tran hai   NaN   
2          US       5  2023-02-27  7.2.0e             Della   NaN   
3          US       5  2023-02-27  7.2.0e     AYNGARAN __02   NaN   
4          US       2  2023-02-27  7.2.0e  vikash gaming op   NaN   
...       ...     ...         ...     ...               ...   ...   
99995      US       5  2018-06-19  3.7.0i     A Google user   NaN   
99996      US       5  2018-06-19  3.7.0i  skarkyboy really   NaN   
99997      US       4  2018-06-19  3.7.0i       Mike Keeler   NaN   
99998      US       5  2018-06-19  3.7.0i     Daquan Nelson   NaN   
99999      US       5  2018-06-19  3.7.0i     moomoo cowies   NaN   

                                                 Content  
0        it's a good way to help u understand what to do  
1      The game is real

In [5]:
from gensim.models import Word2Vec
import nltk
from gensim.models import KeyedVectors
from nltk.cluster import KMeansClusterer
import numpy as np 

from sklearn import cluster
from sklearn import metrics

In [6]:
import re

In [7]:
def nlp_pipeline(text):

    text = text.lower() # mettre les mots en minuscule

# Retirons les caractères spéciaux :

    text = re.sub(r"[,\!\?\%\(\)\/\"]", "", text)
    text = re.sub(r"\&\S*\s", "", text)
    text = re.sub(r"\-", "", text)
    
    return text


In [8]:
!python -m spacy download en_core_web_md

2023-03-07 14:32:11.006521: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 14:32:11.006592: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 14:32:12.522602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-07 14:32:12.522660: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-07 14:32:12.522697: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyther): /proc/driver/nvidia/version does not exist
Defaulting to user installation because normal site-packages is not 

In [9]:
import spacy

In [10]:
pip install spacy 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install spacytextblob

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
!python -m spacy download en_core_web_md

2023-03-07 14:32:26.845725: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 14:32:26.845794: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 14:32:28.320484: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-07 14:32:28.320539: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-07 14:32:28.320575: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyther): /proc/driver/nvidia/version does not exist
Defaulting to user installation because normal site-packages is not 

In [13]:
from spacytextblob.spacytextblob import SpacyTextBlob

In [14]:
nlp =spacy.load("en_core_web_md")

In [15]:
nlp.add_pipe('spacytextblob')

In [16]:
def preprocess_comment(comment):
    doc = nlp(comment.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)


In [17]:
preprocessed_comments = [preprocess_comment(comment) for comment in DML["Content"].iloc[:50]]
print(preprocessed_comments)

['good way help u understand', 'game bad return game game datum go time gameloft game lose datum gameloft want kick veteran player', 'good game pass world', 'madmax', 'star ad annoying remove ad game good', 'dragon eat fruit eat fish meat honeycomb mainly fish fish game', 'good', 'start play log year progress completely wipe talk support respond good backup server', 'love game', 'fun play exit kid adault age age allow download game download dragon mainia ledgends', 'nice', 'easy got gold coin easy got germ', 'game absolutely fun cute opinion thing bit cheap minor thing wait developer come fun play pep', 'love play delete reason account', 'nice game', 'rate star gameplay amazing game pay win need spend heavily obtain dragon difficult obtain normally force pay event pay win need spend heavily thousand participate event pay recommend', 'well dragon legend', 'think great game dragon city', 'ad base game show completely different game star game fun add show able tap object daily life breed 

In [20]:
# Analyser chaque commentaire et afficher le sentiment
for commentaire in preprocessed_comments:
    doc = nlp(commentaire)
    sentiment = doc._.polarity
    if sentiment > 0:
        print(f"{commentaire} -> Sentiment positif ({sentiment})")
    elif sentiment < 0:
        print(f"{commentaire} -> Sentiment négatif ({sentiment})")
    else:
        print(f"{commentaire} -> Sentiment neutre ({sentiment})")

good way help u understand -> Sentiment positif (0.7)
game bad return game game datum go time gameloft game lose datum gameloft want kick veteran player -> Sentiment négatif (-0.3833333333333333)
good game pass world -> Sentiment positif (0.14999999999999997)
madmax -> Sentiment neutre (0.0)
star ad annoying remove ad game good -> Sentiment négatif (-0.16666666666666674)
dragon eat fruit eat fish meat honeycomb mainly fish fish game -> Sentiment négatif (-0.11666666666666668)
good -> Sentiment positif (0.7)
start play log year progress completely wipe talk support respond good backup server -> Sentiment positif (0.39999999999999997)
love game -> Sentiment positif (0.04999999999999999)
fun play exit kid adault age age allow download game download dragon mainia ledgends -> Sentiment négatif (-0.05000000000000002)
nice -> Sentiment positif (0.6)
easy got gold coin easy got germ -> Sentiment positif (0.43333333333333335)
game absolutely fun cute opinion thing bit cheap minor thing wait dev